## Importing Required Libraries

This section imports all the necessary Python libraries and modules used throughout the notebook.  
These include:
- **CARLA** for simulation and environment interaction  
- **NumPy, OpenCV, and Matplotlib** for numerical computations and visualization  
- **Dubins and heapq** for path planning algorithms  
- **cvxpy, osqp, and CasADi** for optimization and control tasks  
- **SciPy** for numerical optimization and sparse matrix operations  

Optional import paths for CARLA’s internal route planner are also provided (commented out).


In [ ]:
# ================================================
# Importing Required Libraries and Modules
# ================================================

# CARLA simulator API for autonomous driving simulations
import carla

# System and file management utilities
import os
import sys
import time
import random

# Numerical and mathematical tools
import numpy as np
import math

# Computer vision and plotting libraries
import cv2
import matplotlib.pyplot as plt

# Path planning algorithms
import dubins      # For Dubins path generation (minimum turning radius)
import heapq       # For priority queue operations (e.g., A* search)

# Optimization and control libraries
import cvxpy as cp         # Convex optimization
import osqp                # Quadratic programming solver
from scipy import sparse   # Sparse matrix utilities
from scipy.optimize import minimize  # General-purpose optimizer

## Connecting to CARLA Simulator and Setting Up the Environment

This section connects to the CARLA simulator running locally and loads the desired map (`Town02`).  
It also configures the **spectator camera** to view the scene from above and retrieves map information such as road topology.  
These steps initialize the simulation environment before spawning vehicles, sensors, or running control logic.


In [ ]:
# ================================================
# Connect to CARLA Simulator and Load Map
# ================================================

# Connect to the CARLA server running on localhost at port 2000
client = carla.Client('localhost', 2000)

# (Optional) Set a timeout for client-server communication in seconds
# client.set_timeout(2.0)

# Load the desired CARLA map (Town02)
world = client.load_world('Town02')

# ================================================
# Retrieve Map and Spectator Information
# ================================================

# Get spectator (camera) object to adjust the viewpoint
spectator = world.get_spectator()

# Get map and road network data
Map = world.get_map()
roads = Map.get_topology()

# ================================================
# Set Spectator (Camera) Viewpoint
# ================================================

# Define a top-down spectator position and orientation
spectator_pos = carla.Transform(
    carla.Location(x=100, y=200, z=150),   # Position in 3D space
    carla.Rotation(pitch=-90, yaw=-90)     # Looking straight down
)

# Apply the transform to the spectator to update the view
spectator.set_transform(spectator_pos)

## Removing Buildings from the Simulation Environment

This section extracts all **building objects** from the loaded CARLA world and disables them.  
Removing buildings helps create an open environment — for instance, when testing path planning, localization, or mapping algorithms without visual obstructions.


In [ ]:
# ================================================
# Remove Buildings from the Simulation Environment
# ================================================

# Extract all building objects present in the current CARLA world
env_objs = world.get_environment_objects(carla.CityObjectLabel.Buildings)

# Collect the unique IDs of all detected building objects
building_ids = {obj.id for obj in env_objs}

# Disable (hide) all buildings using their IDs
# This effectively clears the map for better visibility or simplified testing
world.enable_environment_objects(building_ids, False)


## Generating and Filtering Waypoints from the Map

This section generates waypoints across the CARLA map at **5-meter intervals** and filters them to ensure uniqueness.  
It avoids duplicate or overlapping waypoints by checking spatial and orientation proximity.  
Finally, it selects waypoints belonging to **lane 1** and groups them by their **road IDs** into a dictionary called `Sections`.  

This process creates a clean set of representative waypoints for path planning and visualization.


In [ ]:
# ================================================
# Generate and Filter Waypoints from Map
# ================================================

# Generate all waypoints spaced every 5 meters along the road network
all_waypoints = Map.generate_waypoints(5)

# Initialize containers for unique and filtered waypoints
unique_waypoints = []  # To store non-duplicate waypoints
Sections = {}          # Dictionary grouping waypoints by road_id
SelectedWp = []        # Final list of selected lane 1 waypoints

# --------------------------------
# Remove Duplicate / Overlapping Waypoints
# --------------------------------
for wp in all_waypoints:
    if len(unique_waypoints) == 0:
        unique_waypoints.append(wp)
    else:
        found = False
        for uwp in unique_waypoints:
            # Check spatial and orientation similarity thresholds
            if (abs(uwp.transform.location.x - wp.transform.location.x) < 0.1 and
                abs(uwp.transform.location.y - wp.transform.location.y) < 0.1 and
                abs(uwp.transform.rotation.yaw - wp.transform.rotation.yaw) < 20):
                found = True
                break
        # Add waypoint only if it's sufficiently different
        if not found:
            unique_waypoints.append(wp)

# --------------------------------
# Filter and Organize Waypoints by Lane and Road
# --------------------------------
for wp in unique_waypoints:
    # Only select waypoints from lane 1
    if wp.lane_id == 1:
        SelectedWp.append(wp)
        
        # Group waypoints by road ID into Sections dictionary
        if wp.road_id not in Sections:
            Sections[wp.road_id] = [wp]
        else:
            Sections[wp.road_id].append(wp)

# --------------------------------
# Visualize all waypoints in CARLA world
# Uncomment below to draw all waypoints in the simulator
# --------------------------------
# for wp in all_waypoints:
#     world.debug.draw_string(
#         wp.transform.location, '^', draw_shadow=False, 
#         color=carla.Color(r=0, g=0, b=255), 
#         life_time=80.0, persistent_lines=True
#     )


## Generating a Continuous Path from Spawn Point

This section creates a continuous driving path starting from the **first spawn point** on the map.  
The path is generated by iteratively moving forward from one waypoint to the next, spaced at **5-meter intervals**, until a maximum total path length of **480 meters** is reached.  

The result is a list of waypoints (`path`) that can later be used for vehicle navigation, visualization, or trajectory tracking.  
An optional visualization step (commented out) can be used to display the waypoints in CARLA.  
Finally, the spectator (camera) is reset to a top-down view for better visualization.


In [ ]:
# ================================================
# Generate a Continuous Path Starting from a Spawn Point
# ================================================

# Retrieve the first available spawn point in the CARLA world
spawn_point = world.get_map().get_spawn_points()[0]

# Get the waypoint corresponding to the spawn point location
# 'project_to_road=True' ensures the waypoint is on a drivable lane
start_wp = Map.get_waypoint(
    spawn_point.location, 
    project_to_road=True, 
    lane_type=carla.LaneType.Driving
)

# --------------------------------
# Initialize Path Generation Parameters
# --------------------------------
path = []             # List to store the generated path waypoints
currWp = start_wp     # Start from the initial waypoint
totalLength = 0.0     # Track total path length (in meters)
maxLength = 480       # Maximum length of path to generate
distance = 5          # Distance between consecutive waypoints (in meters)

# --------------------------------
# Generate Path by Chaining Waypoints
# --------------------------------
while currWp and totalLength < maxLength:
    path.append(currWp)                   # Store current waypoint
    next_wps = currWp.next(distance)      # Retrieve next waypoint(s) ahead

    # Stop if no valid continuation exists (e.g., road ends)
    if not next_wps:
        break

    # Select the first available next waypoint
    next_wp = next_wps[0]

    # Update total path length and move to next waypoint
    totalLength += distance
    currWp = next_wp

# --------------------------------
# Visualize the Path in CARLA
# --------------------------------
# for wp in path:
#     world.debug.draw_string(
#         wp.transform.location, '^', draw_shadow=False, 
#         color=carla.Color(r=0, g=0, b=255), 
#         life_time=80.0, persistent_lines=True
#     )

# --------------------------------
# Reset Spectator View (Top-Down Camera)
# --------------------------------
spectator_pos = carla.Transform(
    carla.Location(x=100, y=200, z=150),
    carla.Rotation(pitch=-90, yaw=-90)
)
spectator.set_transform(spectator_pos)


## Organizing Path Waypoints by Road Section

This section groups the generated path waypoints into a dictionary called `PathSections`,  
where each **road ID** serves as a key and its corresponding list of **waypoints** forms the value.  

This organization makes it easier to analyze or visualize the path in terms of separate road segments,  
which can be useful for navigation, segment-based control, or route analysis.


In [ ]:
# ================================================
# Group Path Waypoints by Road Section
# ================================================

# Initialize an empty dictionary to store waypoints by road_id
PathSections = {}

# Iterate through all waypoints in the generated path
for wp in path:
    # If this road_id hasn't been added yet, create a new entry
    if wp.road_id not in PathSections:
        PathSections[wp.road_id] = [wp]
    # Otherwise, append the waypoint to the existing list for that road
    else:
        PathSections[wp.road_id].append(wp)

# Print the dictionary to verify the grouping result
print(PathSections)

## 📐 Utility Functions for Pose and Angle Normalization

This section defines two helper functions used throughout the MPC and path generation pipeline:

1. **`pose_from_transform(wp)`** — Extracts the (x, y, yaw) pose from a CARLA waypoint's transform object.  
   - Converts yaw from degrees to radians for consistency with trigonometric calculations.

2. **`normalize_angle(theta)`** — Normalizes any input angle to the range \([-π, π]\), ensuring continuity of orientation values during path following or optimization.


In [ ]:
def pose_from_transform(wp):
    """
    Extracts the (x, y, yaw) pose from a CARLA waypoint.

    Args:
        wp: CARLA Waypoint object containing a transform (position + rotation)

    Returns:
        (x, y, yaw): Tuple of position and yaw angle (in radians)
    """
    x = wp.transform.location.x
    y = wp.transform.location.y
    yaw = math.radians(wp.transform.rotation.yaw)  # convert to radians
    return (x, y, yaw)


def normalize_angle(theta):
    """
    Normalize an angle to the range [-π, π].

    Args:
        theta: Input angle (in radians)

    Returns:
        Normalized angle (in radians)
    """
    while theta > math.pi:
        theta -= 2.0 * math.pi
    while theta < -math.pi:
        theta += 2.0 * math.pi
    return theta


## Generating a Continuous Dubins Path Across All Road Sections

This section constructs a smooth, drivable path along the map using **Dubins curves**,  
which model the shortest possible path between two poses (x, y, yaw) given a fixed **minimum turning radius**.  

For each road section:
1. A **Dubins path** is generated between the start and end waypoints.
2. Each path is sampled at 1-meter intervals to produce a sequence of (x, y, yaw) points.
3. Between consecutive sections, **connector paths** are created to ensure continuity.  

The result, stored in `FullDubins`, is a **stitched global trajectory** with smooth curvature transitions — suitable for control, planning, or visualization.


In [ ]:

# ================================================
# Generate Continuous Dubins Path for Entire Route
# ================================================

# Define motion constraints and sampling parameters
turning_radius = 5   # Minimum turning radius of the vehicle (meters)
step_size = 1.0        # Step distance between consecutive Dubins samples (meters)

# Container to hold the full stitched Dubins path
FullDubins = []

# Extract ordered list of road section keys
section_keys = list(PathSections.keys())

# --------------------------------
# Generate Dubins Path for Each Road Section
# --------------------------------
for idx, Sec in enumerate(section_keys):
    # Get the start and end poses (x, y, yaw) for this road section
    start = pose_from_transform(PathSections[Sec][0])
    goal = pose_from_transform(PathSections[Sec][-1])

    # Compute the shortest Dubins path between start and goal
    DubPath = dubins.shortest_path(start, goal, turning_radius)

    # Sample the path at regular step intervals
    configurations, _ = DubPath.sample_many(step_size)

    # Extract x, y, yaw directly from Dubins output
    xs   = [q[0] for q in configurations]
    ys   = [q[1] for q in configurations]
    yaws = [normalize_angle(q[2]) for q in configurations]  # Normalize to [-pi, pi]

    # Append this section’s Dubins points to the full trajectory
    for i in range(len(xs)):
        FullDubins.append((xs[i], ys[i], yaws[i]))

    # --------------------------------
    # Connect to the Next Section (if any)
    # --------------------------------
    if idx < len(section_keys) - 1:
        # Get the starting pose of the next section
        next_start = pose_from_transform(PathSections[section_keys[idx + 1]][0])

        # Generate a connecting Dubins path between current and next sections
        connector = dubins.shortest_path((xs[-1], ys[-1], yaws[-1]), next_start, turning_radius)
        configs, _ = connector.sample_many(step_size)

        # Extract x, y, yaw directly from the connector
        cx    = [q[0] for q in configs]
        cy    = [q[1] for q in configs]
        cyaws = [normalize_angle(q[2]) for q in configs]

        # Append connector segment to the full trajectory
        for i in range(len(cx)):
            FullDubins.append((cx[i], cy[i], cyaws[i]))

# --------------------------------
# Print summary of generated trajectory
# --------------------------------
print(f"Generated stitched Dubins path with {len(FullDubins)} waypoints.")

# --------------------------------
# Visualize Dubins Path in CARLA World
# --------------------------------
# for (x, y, yaw) in FullDubins:
#     loc = carla.Location(x=x, y=y, z=1)
#     world.debug.draw_point(
#         loc, size=0.1,
#         color=carla.Color(r=0, g=0, b=255),
#         life_time=30.0
#     )


## Generating Velocity Reference Profile Along the Dubins Path

This section defines a function `generate_vref()` that computes a **velocity reference profile**  
for each point along the generated Dubins path, considering both **curvature** and **acceleration limits**.

The algorithm works as follows:
1. **Curvature Calculation** — Estimates curvature (`kappa`) from yaw changes along the path.  
2. **Lateral Acceleration Limit** — Computes the maximum feasible speed around curves.  
3. **Backward Pass** — Ensures safe **deceleration** when approaching curves or stops.  
4. **Forward Pass** — Ensures safe **acceleration** without exceeding longitudinal limits.  
5. **Velocity Assignment** — Combines results into a list of (x, y, yaw, v_ref) tuples.

Finally, it generates the `FullDubinsV` list at a maximum target speed of **150 km/h** with realistic dynamic constraints.


In [ ]:
# ================================================
# Generate Velocity Reference Profile Along Path
# ================================================

def generate_vref(path_xyyaw, vmax_kmh=40.0, a_lat_max=4.0, a_long_max=2.0):
    path_xyyaw = np.array(path_xyyaw)
    N = len(path_xyyaw)
    vmax = vmax_kmh / 3.6

    kappa = np.zeros(N)
    for i in range(1, N - 1):
        psi_prev = path_xyyaw[i - 1, 2]
        psi_next = path_xyyaw[i + 1, 2]
        ds = np.hypot(path_xyyaw[i + 1, 0] - path_xyyaw[i - 1, 0],
                      path_xyyaw[i + 1, 1] - path_xyyaw[i - 1, 1])
        if ds > 1e-6:
            kappa[i] = (psi_next - psi_prev) / ds
    kappa[0] = kappa[1]
    kappa[-1] = kappa[-2]

    # smoother curvature-limited speed (avoid huge spikes)
    v_curve = np.minimum(vmax, np.sqrt(a_lat_max / (np.abs(kappa) + 1e-4)))
    v_curve = np.convolve(v_curve, np.ones(5)/5, mode='same')

    v_ref = v_curve.copy()

    # Backward and forward passes (with proper start/stop)
    v_ref[-1] = 0.0
    for i in reversed(range(N - 1)):
        ds = np.hypot(path_xyyaw[i + 1, 0] - path_xyyaw[i, 0],
                      path_xyyaw[i + 1, 1] - path_xyyaw[i, 1])
        v_ref[i] = min(v_ref[i], np.sqrt(v_ref[i + 1]**2 + 2 * a_long_max * ds))

    v_ref[0] = 0.0
    for i in range(1, N):
        ds = np.hypot(path_xyyaw[i, 0] - path_xyyaw[i - 1, 0],
                      path_xyyaw[i, 1] - path_xyyaw[i - 1, 1])
        v_ref[i] = min(v_ref[i], np.sqrt(v_ref[i - 1]**2 + 2 * a_long_max * ds))

    FullDubinsV = [(path_xyyaw[i, 0], path_xyyaw[i, 1], path_xyyaw[i, 2], v_ref[i])
                   for i in range(N)]
    return FullDubinsV



# ================================================
# Generate Full Dubins Path with Velocity Profile
# ================================================
FullDubinsV = generate_vref(
    FullDubins, 
    vmax_kmh=5.0,    # Maximum target speed
    a_lat_max=1.5,     # Max lateral acceleration
    a_long_max=1.0     # Max longitudinal acceleration
)


### 🧭 Smooth Lateral Offset Correction for Dubins Path

This section defines a function to **smoothly correct lateral offsets** in the Dubins path.  
It helps ensure the vehicle path remains well-centered and transitions naturally through curves.  

The method detects turning segments based on curvature and applies a **fade-in/fade-out offset** using the turning radius.  
This prevents sudden jumps in lateral displacement and produces a smoother, more realistic trajectory for vehicle motion.


In [ ]:
import numpy as np

def correct_dubins_offset_smooth(pathV, R_min, lane_offset=1.0, fade_len=10, curvature_thresh=0.01):
    """
    Apply smooth, geometrically correct lateral offset to a Dubins path.
    Ensures proper centering on straights while preserving smooth curvature transitions.

    Args:
        pathV : [(x, y, yaw, v_ref), ...]
            List of tuples defining the Dubins path (x [m], y [m], yaw [rad], v_ref [m/s]).
        R_min : float
            Nominal turning radius used for Dubins generation.
        lane_offset : float
            Desired lateral offset magnitude (positive = left, negative = right).
        fade_len : int
            Number of points to fade offset in/out.
        curvature_thresh : float
            Curvature threshold for detecting curves.

    Returns:
        corrected : [(x_corr, y_corr, yaw, v_ref), ...]
            Offset-corrected and smoothed path.
    """

    n = len(pathV)
    corrected = []

    # --- Compute smoothed curvature (yaw rate) ---
    yaw = np.unwrap([p[2] for p in pathV])
    dyaw = np.gradient(yaw)
    curvature = np.convolve(dyaw, np.ones(5)/5, mode='same')

    fade = 0.0
    in_curve = False
    curve_sign = 0.0  # +1 = left turn, -1 = right turn

    for i, (x, y, yaw_i, vref) in enumerate(pathV):
        k = curvature[i]

        # --- Detect curve entry/exit ---
        if abs(k) > curvature_thresh:
            # Inside or entering curve
            if not in_curve:
                curve_sign = np.sign(k)  # lock turning direction
            in_curve = True
            fade = min(fade + 1.0 / fade_len, 1.0)
        else:
            # Exiting curve
            if in_curve:
                fade = max(fade - 1.0 / fade_len, 0.0)
                if fade <= 0.0:
                    in_curve = False
                    curve_sign = 0.0
            else:
                # Stay at zero offset on long straights
                fade = max(fade - 1.0 / fade_len, 0.0)
                curve_sign = 0.0

        # --- Local normal vector (perpendicular to heading) ---
        nx, ny = -np.sin(yaw_i), np.cos(yaw_i)

        # --- Compute final lateral offset ---
        #  - Curves → gradually apply signed offset
        #  - Straights → offset fades to zero
        offset = lane_offset * fade * curve_sign

        # --- Apply offset to (x, y) ---
        x_corr = x + offset * nx
        y_corr = y + offset * ny

        corrected.append((x_corr, y_corr, yaw_i, vref))

    return corrected


# === Example usage ===
R_min = 4.0
lane_offset = 1.0  # lane center offset magnitude
FullDubinsV_centered = correct_dubins_offset_smooth(FullDubinsV, R_min, lane_offset=lane_offset)

# === Optional visualization in CARLA ===
for (x, y, yaw, _) in FullDubinsV_centered:
    loc = carla.Location(x=x, y=y, z=1)
    world.debug.draw_point(
        loc,
        size=0.1,
        color=carla.Color(r=0, g=255, b=255),
        life_time=160.0
    )

print(f"✅ Applied smooth lateral offset correction to Dubins path ({len(FullDubinsV_centered)} points).")


### 🚗 Vehicle Spawning and Cleanup in CARLA

This section removes any existing vehicles from the CARLA world and spawns a new **Tesla Model 3** at a chosen spawn point.  
It ensures a clean simulation environment before adding the new vehicle actor.  

The spectator (camera) view is also reset to a predefined position for better visualization of the scene.


In [ ]:
# --- Remove all existing vehicles to ensure a clean simulation state ---
actors = world.get_actors()
vehicles = actors.filter('vehicle.*')

for vehicle in vehicles:
    vehicle.destroy()  # Safely remove each existing vehicle from the world

# --- Spawn a new Tesla Model 3 at the chosen spawn point ---
vehicle_bp = world.get_blueprint_library().filter("*model3*")  # Fetch vehicle blueprint
vehicle = world.try_spawn_actor(vehicle_bp[0], spawn_point)     # Attempt to spawn the vehicle

# --- Reset the spectator (camera) position for visualization ---
spectator.set_transform(spectator_pos)

### 🎥 Third-Person Camera Setup

This section attaches a **third-person RGB camera** to the vehicle to capture visuals from behind and slightly above.  
The camera is configured with a specific resolution and field of view for optimal observation.  

A callback function processes each captured frame, converting the raw data from CARLA into a NumPy array for further use,  
such as visualization or computer vision processing.


In [ ]:
# === Third-person camera setup ===
# Create a camera blueprint for an RGB sensor
camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '800')  # Image width in pixels
camera_bp.set_attribute('image_size_y', '600')  # Image height in pixels
camera_bp.set_attribute('fov', '90')            # Field of view in degrees

# Define the camera position relative to the vehicle:
# Placed 8 meters behind and 3 meters above, angled slightly downward.
spawn_point = carla.Transform(
    carla.Location(x=-8.0, z=3.0),
    carla.Rotation(pitch=-15)
)

# Attach the camera to the vehicle at the defined position
chase_cam = world.spawn_actor(camera_bp, spawn_point, attach_to=vehicle)

# Shared dictionary to store the latest camera frame
frame_data = {"image": None}

def camera_callback(image):
    """
    Callback function that processes incoming camera frames.
    Converts CARLA's raw BGRA image data into a NumPy RGB array.
    """
    # Convert the raw image buffer to a NumPy array
    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    array = array.reshape((image.height, image.width, 4))  # Reshape to (H, W, BGRA)
    
    # Store only the RGB channels (discard alpha)
    frame_data["image"] = array[:, :, :3]

# Start listening for incoming camera frames
chase_cam.listen(lambda image: camera_callback(image))


### 💾 Video Recording Setup for Simulation Output

This section initializes a **video writer** to record frames captured from the third-person camera.  
It creates an output directory, sets up the video parameters such as resolution, codec, and frame rate,  
and prepares an `.mp4` file to store the recorded CARLA simulation footage.  

This enables playback and post-analysis of the MPC (Model Predictive Control) run or vehicle behavior.


In [ ]:
# === Video recording setup ===

# Define the output directory for recorded videos
output_dir = "videos"
os.makedirs(output_dir, exist_ok=True)  # Create directory if it doesn't exist

# Define the full output video path and filename
video_filename = os.path.join(output_dir, "mpc_run_output_2.mp4")

# Set video dimensions (must match the camera's resolution)
frame_width, frame_height = 800, 600

# Define frames per second based on simulation time step (dt)
# Example: if dt = 0.1s, fps = 10
dt = 0.1
fps = int(1 / dt)

# Define video codec and initialize the video writer for MP4 output
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(video_filename, fourcc, fps, (frame_width, frame_height))

# Confirm setup
print(f"🎥 Video recording initialized: {video_filename}")


In [ ]:
# === Helper function: Compute Cross-Track Error ===
def compute_cte(x, y, path, j, N):
    """
    Compute perpendicular distance from current (x, y)
    to the nearest segment in the reference path horizon.
    """
    cte_min = float('inf')

    # Limit to local segment of the path (for efficiency)
    end_idx = min(j + N, len(path) - 1)
    for i in range(j, end_idx):
        x1, y1 = path[i][0], path[i][1]
        x2, y2 = path[i + 1][0], path[i + 1][1]

        # Vector projection of vehicle onto the segment
        dx, dy = x2 - x1, y2 - y1
        if dx == 0 and dy == 0:
            continue
        u = ((x - x1) * dx + (y - y1) * dy) / (dx**2 + dy**2)
        u = np.clip(u, 0.0, 1.0)

        # Closest point on path segment
        x_closest = x1 + u * dx
        y_closest = y1 + u * dy

        dist = np.sqrt((x - x_closest)**2 + (y - y_closest)**2)
        if dist < cte_min:
            cte_min = dist

    return cte_min

def find_nearest_index(x, y, path):
    dists = [(x - p[0])**2 + (y - p[1])**2 for p in path]
    return int(np.argmin(dists))

### 🧩 Model Predictive Control (MPC) with Acceleration and Jerk Smoothing

This section implements the **core MPC (Model Predictive Control)** logic used to track the planned Dubins path in CARLA.  
The MPC dynamically computes the **optimal steering and acceleration commands** that minimize tracking errors,  
while ensuring smooth and realistic vehicle motion through constraints and penalties.

#### Key Features:
- Predicts vehicle trajectory over a **finite horizon (N)** using the bicycle kinematic model.  
- Penalizes:
  - Position, yaw, and speed tracking errors  
  - Steering angle and steering rate  
  - Lateral acceleration (stability constraint)  
  - Acceleration magnitude and **jerk** (change in acceleration) for smooth control  
- Integrates with CARLA’s synchronous mode for consistent timing.  
- Logs all performance metrics and saves a **video recording** of the run.  
- Includes throttle rate limiting to simulate actuator smoothing.  

This section is the heart of the control system that makes the simulated vehicle follow the planned path robustly.


In [ ]:
# === Parameters (modified) ===
L = 0.25
L_f = L / 2     # front half of wheelbase
L_r = L / 2     # rear half
dt = 0.1
N = 8 #12          # MPC horizon length (number of prediction steps)
max_steer_deg = 15
max_accel = 2  # reduced from 3.0 m/s² for smoother acceleration


# Vehicle physical constants for energy model
mass = 3.2        # kg (RC car mass)
C_r = 0.015       # rolling resistance coefficient
rho = 1.225       # air density (kg/m³)
C_d = 0.4         # drag coefficient
A = 0.02          # frontal area (m²)
g = 9.81


# Use the smoothed, centered Dubins path
path = FullDubinsV  # [(x, y, yaw, v_ref_kmh), ...]

# Convert reference velocities from km/h to m/s
# path = [(x, y, yaw, v_kmh / 3.6) for (x, y, yaw, v_kmh) in path]

# === Enable Carla synchronous mode for deterministic simulation ===
settings = world.get_settings()
settings.synchronous_mode = True
settings.fixed_delta_seconds = dt
world.apply_settings(settings)

# === MPC Solver with acceleration and jerk penalties ===
def mpc_step(x0, y0, psi0, v0, ref_path):
    """
    Compute optimal steering and acceleration using MPC.
    Control vector u = [delta0, a0, delta1, a1, ..., deltaN-1, aN-1]
    """

    def cost(u):
        x, y, psi, v = x0, y0, psi0, v0
        total_cost = 0.0

        # --- Weighting parameters for cost terms ---
        pos_error_weight = 1.5
        yaw_error_weight = 0.8
        speed_error_weight = 0.6
        steer_weight = 0.5
        steer_rate_weight = 20
        lat_accel_weight = 0.8
        accel_mag_weight = 1.5
        accel_smooth_weight = 6.0

        # 🔋 New: Energy-awareness term weight (tune this)
        energy_weight = 0.005 #/ (mass ** 2)  # start small (0.01–0.05), increase if you want smoother driving

        M = len(ref_path) - 1
        if u.shape[0] != 2 * M:
            return 1e9  # invalid control vector size → large penalty

        for k in range(M):
            delta = u[2 * k]
            a = u[2 * k + 1]

            # --- Vehicle dynamics ---
            a_clipped = np.clip(a, -max_accel, max_accel)
            v = v + a_clipped * dt
            beta = np.arctan((L_r / (L_f + L_r)) * np.tan(delta))
            psi = psi + (v / L_r) * np.sin(beta) * dt
            x = x + v * np.cos(psi + beta) * dt
            y = y + v * np.sin(psi + beta) * dt

            # --- Reference tracking errors ---
            px_ref, py_ref, yaw_ref, v_ref = ref_path[k]
            pos_error = pos_error_weight * ((x - px_ref)**2 + (y - py_ref)**2)
            yaw_diff = np.arctan2(np.sin(psi - yaw_ref), np.cos(psi - yaw_ref))
            yaw_error = yaw_error_weight * (yaw_diff**2)
            speed_error = speed_error_weight * (v - v_ref)**2

            # --- Steering and lateral dynamics penalties ---
            steer_pen = steer_weight * (delta**2)
            if k > 0:
                steer_rate = delta - u[2 * (k - 1)]
                steer_pen += steer_rate_weight * (steer_rate**2)

            a_lat = v**2 / L * np.tan(delta)
            lat_pen = lat_accel_weight * (a_lat**2)

            # --- Acceleration penalties ---
            accel_mag_pen = accel_mag_weight * (a_clipped**2)
            accel_smooth_pen = 0.0
            if k > 0:
                a_prev = u[2 * (k - 1) + 1]
                jerk = a - a_prev
                accel_smooth_pen = accel_smooth_weight * (jerk**2)

            # ==============================================================
            # Energy-aware cost term
            # This discourages high-power actions (saves battery)
            # ==============================================================
            F_r = mass * g * C_r
            F_d = 0.5 * rho * C_d * A * v**2
            F = mass * a_clipped + F_r + F_d
            P = F * abs(v)  # instantaneous power [W]
            energy_penalty = energy_weight * (P**2)
            # ==============================================================

            # --- Accumulate total cost ---
            total_cost += (
                pos_error + yaw_error + speed_error +
                steer_pen + lat_pen +
                accel_mag_pen + accel_smooth_pen 
                # + energy_penalty  # 🔋 include here
            )

            # small regularization
            total_cost += 1e-4 * (delta**2 + a**2)

        return total_cost

    # --- Initialize optimization ---
    M = len(ref_path) - 1
    u0 = np.zeros(2 * M)
    bounds = []

    for _ in range(M):
        bounds.append((-np.radians(max_steer_deg), np.radians(max_steer_deg)))
        bounds.append((-max_accel, max_accel))

    res = minimize(cost, u0, bounds=bounds, method='SLSQP',
                   options={'maxiter': 200, 'ftol': 1e-4})

    if not res.success:
        return 0.0, 0.0
    return res.x[0], res.x[1]

# === Initialize data logs for analysis ===
log_x, log_y, log_psi, log_v = [], [], [], []
log_delta, log_accel = [], []
log_throttle, log_brake = [], []
log_cte, log_v_err, log_cost = [], [], []
log_v_ref = []          
log_solve_time = []    

# Power and energy logging
log_power, log_energy = [], []
total_energy = 0.0

print("🚗 Starting MPC control loop with improved accel smoothing...")

# Throttle low-pass filter
prev_throttle = 0.0
throttle_alpha = 0.4  # smoothing factor

# === Main MPC Control Loop ===
while True:
    # --- Get current vehicle state ---
    transform = vehicle.get_transform()
    vel = vehicle.get_velocity()
    x0, y0 = transform.location.x, transform.location.y
    psi0 = math.radians(transform.rotation.yaw)
    v0 = math.sqrt(vel.x**2 + vel.y**2 + vel.z**2)

    # --- Find nearest path index ---
    Near = find_nearest_index(x0, y0, path)
    if Near >= len(path) - N:
        print("🏁 Reached end of path.")
        break

    # --- Define local prediction horizon ---
    horizon = path[Near:Near + N]

  
    t_start = time.time()
    delta_cmd, a_cmd = mpc_step(x0, y0, psi0, v0, horizon)
    t_solve = time.time() - t_start
    log_solve_time.append(t_solve)  

    steer_norm = np.clip(delta_cmd, -np.radians(max_steer_deg), np.radians(max_steer_deg)) / np.radians(max_steer_deg)

    # --- Convert acceleration command to throttle/brake ---
    raw_throttle = np.clip(max(a_cmd, 0) / max_accel, 0.0, 1.0)
    raw_brake = np.clip(max(-a_cmd, 0) / max_accel, 0.0, 1.0)

    # Apply throttle smoothing
    throttle = prev_throttle * (1 - throttle_alpha) + raw_throttle * throttle_alpha
    prev_throttle = throttle
    brake = raw_brake if raw_brake > 0.01 else 0.0

    # --- Apply control to vehicle ---
    vehicle.apply_control(carla.VehicleControl(
        throttle=float(throttle),
        steer=float(steer_norm),
        brake=float(brake)
    ))

    # --- Advance simulation ---
    world.tick()
    time.sleep(dt)
    if frame_data["image"] is not None:
        frame = frame_data["image"]

        # Convert from BGRA to BGR if needed
        if frame.shape[2] == 4:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGRA2BGR)

        #  Make the frame writable
        frame = np.array(frame, copy=True)

        # === Overlay telemetry data ===
        vel_ms = v0                     # current velocity in m/s
        vel_kmh = v0 * 3.6              # convert to km/h
        accel_ms2 = a_cmd               # acceleration command
        steer_deg = np.degrees(delta_cmd)  # steering in degrees

        telemetry_lines = [
            f"Velocity: {vel_ms:.2f} m/s ({vel_kmh:.1f} km/h)",
            f"Acceleration: {accel_ms2:.2f} m/s²",
            f"Steering (deg): {steer_deg:.1f}"
        ]

        font = cv2.FONT_HERSHEY_SIMPLEX
        scale = 0.6
        color = (0, 0, 0)
        thickness = 2
        y0, dy = 30, 30

        for i, line in enumerate(telemetry_lines):
            y = y0 + i * dy
            cv2.putText(frame, line, (20, y), font, scale, color, thickness, cv2.LINE_AA)

        # === Display and record frame ===
        cv2.imshow("Chase Cam", frame)
        cv2.waitKey(1)
        video_writer.write(frame)

    # --- Energy computation ---
    F_r = mass * g * C_r
    F_d = 0.5 * rho * C_d * A * v0**2
    F = mass * a_cmd + F_r + F_d
    P = F * abs(v0)
    E = P * dt
    total_energy += E

    log_power.append(P)
    log_energy.append(total_energy)

    # --- Logging ---
    ref_x, ref_y, ref_yaw, ref_v = horizon[0]
    cte = compute_cte(x0, y0, path, Near, N)
    v_err = v0 - ref_v
    inst_cost = (2.0 * cte**2) + (0.8 * (psi0 - ref_yaw)**2) + (1.2 * v_err**2) + (0.5 * delta_cmd**2)

    log_x.append(x0); log_y.append(y0); log_psi.append(psi0); log_v.append(v0)
    log_delta.append(delta_cmd); log_accel.append(a_cmd)
    log_throttle.append(throttle); log_brake.append(brake)
    log_cte.append(cte); log_v_err.append(v_err); log_cost.append(inst_cost)
    log_v_ref.append(ref_v) 

    # --- Visualize current target waypoint ---
    tx, ty, _ = path[Near][:3] if not hasattr(path[Near], "transform") else (
        path[Near].transform.location.x,
        path[Near].transform.location.y,
        0
    )
    car_z = transform.location.z
    world.debug.draw_point(carla.Location(x=tx, y=ty, z=car_z + 0.5),
                           size=0.1, color=carla.Color(255, 0, 0), life_time=dt)

print("✅ MPC finished path tracking and data logged successfully.")



# Run baseline (normal MPC)
log_power_energy = log_power.copy()
log_energy_energy = log_energy.copy()
log_v_energy = log_v.copy()
log_cost_energy = log_cost.copy()



# === Final cleanup ===
video_writer.release()
print(" Video saved successfully.")
chase_cam.stop()
chase_cam.destroy()
cv2.destroyAllWindows()



## 📊 MPC Performance Visualization

This section visualizes the performance of the Model Predictive Control (MPC) system after completing the simulation.  
The plots compare reference and actual trajectories, monitor control inputs, and display key performance metrics such as cross-track error, acceleration, and cost evolution.  

Specifically, it includes:
- Reference vs Actual path tracking  
- Cross-track error evolution  
- Speed tracking against reference velocity  
- Steering angle variation  
- Acceleration commands  
- Instantaneous MPC cost over time


In [ ]:
# === Convert logs to numpy arrays for easy plotting ===
log_x, log_y = np.array(log_x), np.array(log_y)
log_v, log_delta, log_accel = np.array(log_v), np.array(log_delta), np.array(log_accel)
log_cte, log_cost = np.array(log_cte), np.array(log_cost)
log_power, log_energy = np.array(log_power), np.array(log_energy)
log_v_ref = np.array(log_v_ref)            
log_solve_time = np.array(log_solve_time)   

# === SAVE LOGS ===
label = "FullRC-Baseline"  # 
save_path = f"logs_{label}.npz"

np.savez(
    save_path,
    x=log_x,
    y=log_y,
    v=log_v,
    v_ref=log_v_ref,      # 
    delta=log_delta,
    accel=log_accel,
    cte=log_cte,
    cost=log_cost,
    power=log_power,
    energy=log_energy,
    solve_time=log_solve_time  # 
)

print(f"✅ Logs saved to {save_path}")

# === Generate summary plots of MPC performance ===
plt.figure(figsize=(18, 16))

# 1️⃣ Path Tracking
plt.subplot(5, 2, 1)
plt.plot([p[0] for p in path], [p[1] for p in path], 'k--', label="Reference Path")
plt.plot(log_x, log_y, 'r', label="Actual Trajectory")
plt.xlabel("X [m]"); plt.ylabel("Y [m]")
plt.title("Path Tracking")
plt.legend()

# 2️⃣ Cross-Track Error
plt.subplot(5, 2, 2)
plt.plot(log_cte, 'b')
plt.xlabel("Timestep"); plt.ylabel("CTE [m]")
plt.title("Cross-Track Error vs Time")

# 3️⃣ Speed Tracking
plt.subplot(5, 2, 3)
plt.plot(log_v, 'r', label="Actual Speed [m/s]")
plt.plot(log_v_ref[:len(log_v)], 'k--', label="Reference Speed [m/s]")
plt.xlabel("Timestep"); plt.ylabel("Speed [m/s]")
plt.legend(); plt.title("Speed Tracking")

# 4️⃣ Steering Command
plt.subplot(5, 2, 4)
plt.plot(np.degrees(log_delta))
plt.xlabel("Timestep"); plt.ylabel("Steering [deg]")
plt.title("Steering Command vs Time")

# 5️⃣ Acceleration Command
plt.subplot(5, 2, 5)
plt.plot(log_accel)
plt.xlabel("Timestep"); plt.ylabel("Acceleration [m/s²]")
plt.title("Acceleration Command vs Time")

# 6️⃣ Instantaneous MPC Cost
plt.subplot(5, 2, 6)
plt.plot(log_cost)
plt.xlabel("Timestep"); plt.ylabel("Cost")
plt.title("Instantaneous MPC Cost Evolution")

# 7️⃣ Instantaneous Power Consumption
plt.subplot(5, 2, 7)
plt.plot(log_power, 'm')
plt.xlabel("Timestep"); plt.ylabel("Power [W]")
plt.title("Instantaneous Power Consumption")

# 8️⃣ Cumulative Energy Usage
plt.subplot(5, 2, 8)
plt.plot(log_energy, 'g')
plt.xlabel("Timestep"); plt.ylabel("Energy [J]")
plt.title("Cumulative Energy Usage Over Time")

# 9️⃣ MPC Solver Time per Step
plt.subplot(5, 2, 9)
plt.plot(log_solve_time, 'c')
plt.xlabel("Timestep"); plt.ylabel("Time [s]")
plt.title("MPC Solver Time per Step")

# 🔟 Solver Time Distribution
plt.subplot(5, 2, 10)
plt.hist(log_solve_time, bins=30, color='gray', edgecolor='black')
plt.xlabel("Solver Time [s]"); plt.ylabel("Frequency")
plt.title("Solver Time Distribution")

plt.tight_layout()
plt.show()

# === Optional summary ===
print(f"⏱️ Mean solver time per MPC step: {np.mean(log_solve_time):.4f} s")
print(f"⏱️ Max solver time per step: {np.max(log_solve_time):.4f} s")


In [ ]:

# === Load both runs ===
# base = np.load("logs_baseline3-newlogs.npz")
base = np.load("logs_FullRC-Baseline.npz")
ea   = np.load("logs_FullRC-EnergyAware.npz")

# === Extract logs ===
log_cte_base, log_cte_ea = base["cte"], ea["cte"]
log_v_base, log_v_ea = base["v"], ea["v"]
log_vref_base, log_vref_ea = base["v_ref"], ea["v_ref"]
log_power_base, log_power_ea = base["power"], ea["power"]
log_energy_base, log_energy_ea = base["energy"], ea["energy"]
log_cost_base, log_cost_ea = base["cost"], ea["cost"]
log_accel_base, log_accel_ea = base["accel"], ea["accel"]

# Optional solver time logs
log_solve_base = base.get("solve_time", None)
log_solve_ea   = ea.get("solve_time", None)

# === Define time axes (based on dt) ===
dt = 0.1  # timestep used in your MPC loop
t_base = np.arange(len(log_v_base)) * dt
t_ea   = np.arange(len(log_v_ea)) * dt

# === Plot comparison ===
plt.figure(figsize=(16, 18))

# 1️⃣ Cross-Track Error (CTE)
plt.subplot(4, 2, 1)
plt.plot(t_base, np.abs(log_cte_base), 'b', label="Baseline MPC")
plt.plot(t_ea, np.abs(log_cte_ea), 'r--', label="Energy-Aware MPC")
plt.xlabel("Time [s]"); plt.ylabel("CTE [m]")
plt.title("Cross-Track Error Comparison")
plt.legend(); plt.grid(True)

# 2️⃣ Speed Tracking
plt.subplot(4, 2, 2)
# plt.plot(t_base, log_vref_base, 'k--', label="Reference Speed Baseline [m/s]")
plt.plot(t_base, log_v_base, 'b', label="Baseline MPC")
plt.plot(t_ea, log_v_ea, 'r--', label="Energy-Aware MPC")
# plt.plot(t_ea, log_vref_ea, 'k--', label="Reference Speed Energy Aware [m/s]")
plt.xlabel("Time [s]"); plt.ylabel("Speed [m/s]")
plt.title("Speed Tracking Comparison")
plt.legend(); plt.grid(True)

# 3️⃣ Acceleration Commands
plt.subplot(4, 2, 3)
plt.plot(t_base, log_accel_base, 'b', label="Baseline MPC")
plt.plot(t_ea, log_accel_ea, 'r--', label="Energy-Aware MPC")
plt.xlabel("Time [s]"); plt.ylabel("Acceleration [m/s²]")
plt.title("Acceleration Command Comparison")
plt.legend(); plt.grid(True)

# 4️⃣ Instantaneous Power
plt.subplot(4, 2, 4)
plt.plot(t_base, log_power_base, 'b', label="Baseline MPC")
plt.plot(t_ea, log_power_ea, 'r--', label="Energy-Aware MPC")
plt.xlabel("Time [s]"); plt.ylabel("Power [W]")
plt.title("Instantaneous Power Comparison")
plt.legend(); plt.grid(True)

# 5️⃣ Cumulative Energy
plt.subplot(4, 2, 5)
plt.plot(t_base, log_energy_base, 'b', label="Baseline MPC")
plt.plot(t_ea, log_energy_ea, 'r--', label="Energy-Aware MPC")
plt.xlabel("Time [s]"); plt.ylabel("Energy [J]")
plt.title("Cumulative Energy Usage Comparison")
plt.legend(); plt.grid(True)

# 6️⃣ MPC Cost
plt.subplot(4, 2, 6)
plt.plot(t_base, log_cost_base, 'b', label="Baseline MPC")
plt.plot(t_ea, log_cost_ea, 'r--', label="Energy-Aware MPC")
plt.xlabel("Time [s]"); plt.ylabel("Cost")
plt.title("Instantaneous MPC Cost Comparison")
plt.legend(); plt.grid(True)

# 7️⃣ Solver Time per Step
if log_solve_base is not None and log_solve_ea is not None:
    plt.subplot(4, 2, 7)
    plt.plot(np.arange(len(log_solve_base)) * dt, log_solve_base, 'b', label="Baseline MPC")
    plt.plot(np.arange(len(log_solve_ea)) * dt, log_solve_ea, 'r--', label="Energy-Aware MPC")
    plt.xlabel("Time [s]"); plt.ylabel("Solver Time [s]")
    plt.title("MPC Solver Time per Step")
    plt.legend(); plt.grid(True)

    plt.subplot(4, 2, 8)
    plt.hist(log_solve_base, bins=30, alpha=0.6, color='blue', label="Baseline")
    plt.hist(log_solve_ea, bins=30, alpha=0.6, color='red', label="Energy-Aware")
    plt.xlabel("Solver Time [s]"); plt.ylabel("Frequency")
    plt.title("Solver Time Distribution")
    plt.legend(); plt.grid(True)
else:
    plt.subplot(4, 2, 7)
    plt.axis('off')
    plt.text(0.5, 0.5, "Solver time logs not available", ha='center', va='center', fontsize=12)

plt.tight_layout()
plt.show()

# === Print solver time summary if available ===
if log_solve_base is not None and log_solve_ea is not None:
    print(f"Baseline: mean solver time = {np.mean(log_solve_base):.4f}s, max = {np.max(log_solve_base):.4f}s")
    print(f"Energy-Aware: mean solver time = {np.mean(log_solve_ea):.4f}s, max = {np.max(log_solve_ea):.4f}s")


In [ ]:
def summarize_power(log_power, log_energy, label="run", dt=0.1):
    log_power = np.array(log_power)
    total_energy = log_energy[-1] if len(log_energy) > 0 else np.sum(log_power) * dt
    pos_mask = log_power > 0
    mean_power = np.mean(log_power)
    mean_pos = np.mean(log_power[pos_mask]) if np.any(pos_mask) else 0.0
    mean_neg = np.mean(log_power[~pos_mask]) if np.any(~pos_mask) else 0.0
    pos_energy = np.sum(log_power[pos_mask]) * dt
    neg_energy = np.sum(log_power[~pos_mask]) * dt

    print(f"--- {label} ---")
    print(f"mean power = {mean_power:.2f} W, mean pos = {mean_pos:.2f}, mean neg = {mean_neg:.2f}")
    print(f"pos energy (J) = {pos_energy:.1f}, neg energy (J) = {neg_energy:.1f}, total energy (J) = {total_energy:.1f}")
    print(f"time steps = {len(log_power)}, total time = {len(log_power)*dt:.1f}s")
    return dict(mean=mean_power, mean_pos=mean_pos, pos_energy=pos_energy, total=total_energy)


# === Run for both baseline and energy-aware ===
dt = 0.1
sum_base = summarize_power(log_power_base, log_energy_base, "Baseline", dt)
sum_ea   = summarize_power(log_power_ea, log_energy_ea, "Energy-Aware", dt)

# === Compute percentage decreases (relative to baseline) ===
perc_mean_power = ((sum_base['mean'] - sum_ea['mean']) / sum_base['mean']) * 100
perc_mean_pos   = ((sum_base['mean_pos'] - sum_ea['mean_pos']) / sum_base['mean_pos']) * 100
perc_energy     = ((sum_base['total'] - sum_ea['total']) / sum_base['total']) * 100

print()
print(f"Mean power decrease: {perc_mean_power:.2f}%")
print(f"Mean positive power decrease: {perc_mean_pos:.2f}%")
print(f"Total energy decrease: {perc_energy:.2f}%")
